In [1]:
%pip install py-algorand-sdk


  Using cached py_algorand_sdk-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-macosx_10_10_universal2.whl (349 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached msgpack-1.0.7-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached cffi-1.16.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Using cached py_algorand_sdk-2.5.0-py3-none-any.whl (111 kB)
Using cached msgpack-1.0.7-cp310-cp310-macosx_11_0_arm64.whl (231 kB)
Using cached pycryptodomex-3.20.0-cp35-abi3-macosx_10_9_universal2.whl (2.4 MB)
Using cached cffi-1.16.0-cp310-cp310-macosx_11_0_arm64.whl (176 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json

In [5]:
import base64
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import *

In [4]:
mnemonic1 = "kiss hope rubber day sphere identify journey material just depend case camp wash like simple dash push tone truck change chicken peace wrong able glove"
mnemonic2 = "soup window nice vapor chimney also sugar cram present dry blame obscure illegal include scheme flame gaze twist crop confirm vault economy hip abandon favorite"
mnemonic3 = "note mix true cream demise inside bench ozone cinnamon frame charge crush into travel ensure blanket rose volume blade humor truly final park abstract boat"

In [7]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    private_key = mnemonic.to_private_key(m)
    accounts[counter]['pk'] = account.address_from_private_key(private_key)
    accounts[counter]['sk'] = private_key
    counter += 1

In [8]:
# Specify the node address and token.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

In [9]:
print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))

Account 1 address: FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI
Account 2 address: OWMS4KE2TTZ6UPBHSODAQKQASVT22LT7KTLSYO4ZHNFEUCFMHC3OHTBOTU


In [62]:
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn
from algosdk.v2client import algod
from algosdk import account, mnemonic, transaction
import json
import time

def wait_for_confirmation(client, transaction_id, timeout=5):
    """
    Wait until the transaction is confirmed.

    Parameters:
    - client: AlgodClient
    - transaction_id: str
    - timeout: int (optional, default: 5)
    """
    start_round = client.status().get("last-round") + 1
    current_round = start_round

    while current_round < start_round + timeout:
        try:
            response = client.pending_transaction_info(transaction_id)
            if response.get("confirmed-round") > 0:
                print(f"Transaction {transaction_id} confirmed in round {response.get('confirmed-round')}.")
                return response.get("confirmed-round")
            elif response.get("pool-error"):
                print(f"Transaction {transaction_id} rejected from the pool.")
                return None
        except Exception as e:
            pass

        client.status_after_block(current_round)
        current_round += 1

    print(f"Transaction {transaction_id} not confirmed after {timeout} rounds.")
    return None

# Specify the node address and token.
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

# Account addresses
account1_address = "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI"
account2_address = "OWMS4KE2TTZ6UPBHSODAQKQASVT22LT7KTLSYO4ZHNFEUCFMHC3OHTBOTU"

# Mnemonics
mnemonic1 = "kiss hope rubber day sphere identify journey material just depend case camp wash like simple dash push tone truck change chicken peace wrong able glove"
mnemonic2 = "soup window nice vapor chimney also sugar cram present dry blame obscure illegal include scheme flame gaze twist crop confirm vault economy hip abandon favorite"

# Convert mnemonics to private keys
private_key1 = mnemonic.to_private_key(mnemonic1)
private_key2 = mnemonic.to_private_key(mnemonic2)

# Get suggested parameters for the transaction
params = algod_client.suggested_params()
params.fee = 1000  # Set an appropriate fee value
time.sleep(1)

# Asset Creation transaction
txn = transaction.AssetConfigTxn(
    sender=account.address_from_private_key(private_key1),
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="latinum",
    manager=account2_address,
    freeze=account2_address,
    clawback=account2_address,
    url="https://path/to/my/asset/details",
    decimals=0,
    strict_empty_address_check=False  # Set strict_empty_address_check to False
)

# Sign with the secret key of the creator
stxn = txn.sign(private_key1)

# Query account information for account 1
account_info_1 = algod_client.account_info(account.address_from_private_key(private_key1))
print("\nAccount 1 information:")
print(json.dumps(account_info_1, indent=4))

# Query account information for account 2
account_info_2 = algod_client.account_info(account.address_from_private_key(private_key2))
print("\nAccount 2 information:")
print(json.dumps(account_info_2, indent=4))


Account 1 information:
{
    "address": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "amount": 0,
    "amount-without-pending-rewards": 0,
    "apps-local-state": [],
    "apps-total-schema": {
        "num-byte-slice": 0,
        "num-uint": 0
    },
    "assets": [],
    "created-apps": [],
    "created-assets": [],
    "min-balance": 100000,
    "pending-rewards": 0,
    "reward-base": 0,
    "rewards": 0,
    "round": 13019,
    "status": "Offline",
    "total-apps-opted-in": 0,
    "total-assets-opted-in": 0,
    "total-created-apps": 0,
    "total-created-assets": 0
}

Account 2 information:
{
    "address": "OWMS4KE2TTZ6UPBHSODAQKQASVT22LT7KTLSYO4ZHNFEUCFMHC3OHTBOTU",
    "amount": 0,
    "amount-without-pending-rewards": 0,
    "apps-local-state": [],
    "apps-total-schema": {
        "num-byte-slice": 0,
        "num-uint": 0
    },
    "assets": [],
    "created-apps": [],
    "created-assets": [],
    "min-balance": 100000,
    "pending-rewards": 0,
  